<a href="https://colab.research.google.com/github/Jyothi-CY/RansomSentry/blob/main/RansomSentry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Ransomware Analysis and Decryption Toolkit
# @markdown Complete toolkit for analyzing and decrypting ransomware

# Install system dependencies for binary analysis
!apt-get update
!apt-get install -y file binutils hexdump xxd

# Install Python libraries for malware analysis
!pip install pefile pycryptodome capstone unicorn

import os
import struct
import hashlib
import binascii
import math
from Crypto.Cipher import AES, DES, ARC4
from Crypto.Util.Padding import unpad
import pefile
import zipfile
import tempfile
from collections import Counter
import string

print("✅ Ransomware Analysis Toolkit Installed Successfully!")

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,288 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,823 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,479 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRe

**Explanation:**

**System Tools:** file, binutils, hexdump, xxd for binary analysis

**Python Libraries:**

pefile: PE file format parsing

pycryptodome: Cryptographic operations

capstone: Disassembly framework

unicorn: CPU emulation

**Core Imports:**
Essential Python libraries for file operations, hashing, and cryptography

In [ ]:
# @title Basic Ransomware Analysis Framework

class RansomwareAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.analysis_results = {}

    def file_type_analysis(self):
        """Analyze file type and basic properties"""
        print("🔍 File Type Analysis")
        print("=" * 50)

        try:
            # Get file info using system commands
            !file "{self.file_path}"
            print("\n")
            !hexdump -C "{self.file_path}" | head -20
        except Exception as e:
            print(f"Error in file analysis: {e}")

    def pe_analysis(self):
        """Analyze PE files (common for Windows ransomware)"""
        print("\n📁 PE File Analysis")
        print("=" * 50)

        try:
            pe = pefile.PE(self.file_path)

            print(f"Machine Type: {pe.FILE_HEADER.Machine}")
            print(f"Number of Sections: {pe.FILE_HEADER.NumberOfSections}")
            print(f"Entry Point: 0x{pe.OPTIONAL_HEADER.AddressOfEntryPoint:x}")

            print("\nSections:")
            for section in pe.sections:
                print(f"  {section.Name.decode().rstrip('\\x00'):8} | "
                      f"Virtual: 0x{section.VirtualAddress:08x} | "
                      f"Raw: 0x{section.PointerToRawData:08x} | "
                      f"Size: 0x{section.SizeOfRawData:08x}")

            print("\nImports (suspicious APIs):")
            suspicious_apis = ['CryptEncrypt', 'CryptDecrypt', 'CreateFile',
                              'WriteFile', 'DeleteFile', 'MoveFile', 'SetFileAttributes']

            for entry in pe.DIRECTORY_ENTRY_IMPORT:
                print(f"  {entry.dll.decode()}")
                for imp in entry.imports:
                    if imp.name and any(api in imp.name.decode() for api in suspicious_apis):
                        print(f"    ⚠️  {imp.name.decode()}")

        except Exception as e:
            print(f"Not a PE file or error in analysis: {e}")

    def string_analysis(self):
        """Extract and analyze strings from the binary"""
        print("\n📝 String Analysis")
        print("=" * 50)

        try:
            # Extract strings using system strings command
            !strings -n 8 "{self.file_path}" > /tmp/strings.txt

            with open('/tmp/strings.txt', 'r') as f:
                strings_data = f.readlines()

            # Look for suspicious patterns common in ransomware
            ransom_indicators = ['.encrypted', '.locked', 'ransom', 'bitcoin',
                               'payment', 'decrypt', 'key', 'aes', 'des', 'rsa']

            print("Suspicious strings found:")
            for line in strings_data:
                line_lower = line.lower()
                if any(indicator in line_lower for indicator in ransom_indicators):
                    print(f"  ⚠️  {line.strip()}")

            # Show most common strings
            print(f"\nTotal strings found: {len(strings_data)}")

        except Exception as e:
            print(f"Error in string analysis: {e}")

    def entropy_analysis(self):
        """Calculate file entropy to detect encryption - FIXED VERSION"""
        print("\n📊 Entropy Analysis")
        print("=" * 50)

        try:
            with open(self.file_path, 'rb') as f:
                data = f.read()

            # Calculate Shannon entropy correctly
            if len(data) == 0:
                entropy = 0
            else:
                counter = Counter(data)
                entropy = 0
                for count in counter.values():
                    p_x = count / len(data)
                    if p_x > 0:  # Avoid log(0)
                        entropy += -p_x * math.log2(p_x)

            print(f"File Entropy: {entropy:.4f}")

            # Interpret entropy values
            if entropy > 7.0:
                print("⚠️  HIGH ENTROPY - Possible encrypted content")
            elif entropy > 6.0:
                print("⚠️  Medium entropy - Possibly compressed or weakly encrypted")
            else:
                print("✅ Low entropy - Likely unencrypted")

        except Exception as e:
            print(f"Error in entropy analysis: {e}")

    def full_analysis(self):
        """Run complete analysis"""
        print(f"🚀 Analyzing: {self.file_path}")
        print("=" * 60)

        self.file_type_analysis()
        self.pe_analysis()
        self.string_analysis()
        self.entropy_analysis()

In [ ]:
# Example usage with a test file
def create_test_file():
    """Create a test file for analysis"""
    # Create a fake PE file with MZ header
    test_data = b'MZ' + b'\x90' * 58 + b'This is a test file for ransomware analysis' + b'\x00' * 100
    with open('/tmp/test_file.exe', 'wb') as f:
        f.write(test_data)
    return '/tmp/test_file.exe'

# Test the analyzer
test_file = create_test_file()
analyzer = RansomwareAnalyzer(test_file)
analyzer.full_analysis()

🚀 Analyzing: /tmp/test_file.exe
🔍 File Type Analysis
/tmp/test_file.exe: MS-DOS executable


00000000  4d 5a 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |MZ..............|
00000010  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000030  90 90 90 90 90 90 90 90  90 90 90 90 54 68 69 73  |............This|
00000040  20 69 73 20 61 20 74 65  73 74 20 66 69 6c 65 20  | is a test file |
00000050  66 6f 72 20 72 61 6e 73  6f 6d 77 61 72 65 20 61  |for ransomware a|
00000060  6e 61 6c 79 73 69 73 00  00 00 00 00 00 00 00 00  |nalysis.........|
00000070  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
*
000000c0  00 00 00 00 00 00 00 00  00 00 00                 |...........|
000000cb

📁 PE File Analysis
Not a PE file or error in analysis: 'Invalid e_lfanew value, probably not a PE file'

📝 String Analysis
Suspicious strings found:
  ⚠️  This is a test file for ransomware analysis

Total strings found: 1

📊 Entropy Analysis
File Entropy: 2.3529
✅ 

**Key Features:**

File Type Analysis: Uses file command and hexdump

PE Analysis: Parses Windows executable structure

String Analysis: Extracts and scans for ransomware indicators

Entropy Analysis: Fixed Shannon entropy calculation to detect encryption

In [ ]:
# @title Fixed Cryptographic Analysis Tools

class CryptoAnalyzer:
    def __init__(self):
        self.encryption_types = {
            'AES': {'block_size': 16, 'entropy_threshold': 7.5},
            'DES': {'block_size': 8, 'entropy_threshold': 7.0},
            'RSA': {'entropy_threshold': 7.8},
            'XOR': {'entropy_threshold': 6.5},
            'Base64': {'entropy_threshold': 5.5}
        }

    def calculate_entropy(self, data):
        """Calculate Shannon entropy of data - CORRECTED VERSION"""
        if len(data) == 0:
            return 0

        # Count frequency of each byte value
        counter = {}
        for byte in data:
            counter[byte] = counter.get(byte, 0) + 1

        # Calculate entropy - FIXED: using math.log2 instead of bit_length
        entropy = 0.0
        for count in counter.values():
            p_x = count / len(data)
            if p_x > 0:  # Avoid log(0)
                entropy += -p_x * math.log2(p_x)

        return entropy

    def detect_encryption_type(self, data):
        """Detect potential encryption type based on data characteristics"""
        if not data:
            return "No data provided"

        entropy = self.calculate_entropy(data)
        data_len = len(data)

        print("🔐 Encryption Type Detection")
        print("=" * 50)
        print(f"Data length: {data_len} bytes")
        print(f"Entropy: {entropy:.4f} bits/byte")

        # Check for common encryption patterns
        results = []

        # Check AES (high entropy, multiple of 16)
        if entropy > 7.5 and data_len % 16 == 0:
            results.append(('AES', 0.9))

        # Check DES (high entropy, multiple of 8)
        if entropy > 7.0 and data_len % 8 == 0:
            results.append(('DES', 0.8))

        # Check RSA (very high entropy)
        if entropy > 7.8:
            results.append(('RSA', 0.85))

        # Check XOR (medium entropy)
        if 6.0 < entropy < 7.5:
            results.append(('XOR', 0.7))

        # Check Base64 (lower entropy, specific pattern)
        if entropy < 6.0 and self._is_likely_base64(data):
            results.append(('Base64', 0.95))

        # Sort by confidence
        results.sort(key=lambda x: x[1], reverse=True)

        if results:
            print("\nDetected encryption types (most likely first):")
            for enc_type, confidence in results:
                print(f"  {enc_type}: {confidence:.1%} confidence")
            return results[0][0]
        else:
            print("\nNo specific encryption type detected")
            return "Unknown"

    def _is_likely_base64(self, data):
        """Check if data has Base64 characteristics"""
        try:
            # Check if data consists mainly of Base64 characters
            base64_chars = set(b'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/=')
            base64_count = sum(1 for byte in data if byte in base64_chars)
            return base64_count / len(data) > 0.95
        except:
            return False

    def brute_force_simple_crypto(self, encrypted_data, max_key_size=10):
        """Attempt to brute force simple encryption methods"""
        print(f"\n🔓 Attempting brute force (max key size: {max_key_size})...")

        results = []

        # Try XOR with different keys
        for key_size in range(1, max_key_size + 1):
            for key in range(256):  # Try all single-byte keys
                decrypted = bytes([b ^ key for b in encrypted_data[:100]])  # Test first 100 bytes

                # Check if result contains mostly printable ASCII
                printable_count = sum(1 for b in decrypted if 32 <= b <= 126)
                if printable_count / len(decrypted) > 0.8:
                    entropy = self.calculate_entropy(decrypted)
                    if entropy < 6.0:  # Lower entropy suggests successful decryption
                        results.append({
                            'method': f'XOR (key: 0x{key:02x})',
                            'key_size': key_size,
                            'entropy': entropy,
                            'sample': decrypted[:20]
                        })

        # Sort by entropy (lower is better for plaintext)
        results.sort(key=lambda x: x['entropy'])

        if results:
            print("Potential decryptions found:")
            for i, result in enumerate(results[:5]):  # Show top 5
                print(f"  {i+1}. {result['method']}, entropy: {result['entropy']:.3f}")
                print(f"     Sample: {result['sample']}")
            return results[0]
        else:
            print("No successful decryptions found")
            return None

In [ ]:
# Example encrypted data (XOR encrypted)
def create_encrypted_example():
    """Create example encrypted data for testing"""
    plaintext = b"This is a secret message that was encrypted using XOR with key 0x42"
    key = b'\x42'
    encrypted = bytes(b ^ key[0] for b in plaintext)
    return encrypted

# Test the crypto analyzer
print("🧪 Testing Crypto Analyzer")
print("=" * 50)

crypto = CryptoAnalyzer()
encrypted_data = create_encrypted_example()

print("Encrypted data (hex):", binascii.hexlify(encrypted_data[:50]))
print()

# Run detection
detected_type = crypto.detect_encryption_type(encrypted_data)
print()

# Run brute force
results = crypto.brute_force_simple_crypto(encrypted_data)

if results:
    print(f"\n✅ Best decryption result:")
    print(f"Method: {results['method']}")
    print(f"Entropy: {results['entropy']:.3f}")
    print(f"Sample: {results['sample']}")

    # Decrypt full message with the found key
    key_byte = int(results['method'].split('0x')[1].split(')')[0], 16)
    full_decrypted = bytes(b ^ key_byte for b in encrypted_data)
    print(f"Full decrypted message: {full_decrypted.decode('ascii')}")

🧪 Testing Crypto Analyzer
Encrypted data (hex): b'162a2b31622b31622362312721302736622f27313123252762362a23366235233162272c21303b323627266237312b2c2562'

🔐 Encryption Type Detection
Data length: 67 bytes
Entropy: 4.1752 bits/byte

No specific encryption type detected


🔓 Attempting brute force (max key size: 10)...
Potential decryptions found:
  1. XOR (key: 0x00), entropy: 4.175
     Sample: b"\x16*+1b+1b#b1'!0'6b/'1"
  2. XOR (key: 0x01), entropy: 4.175
     Sample: b'\x17+*0c*0c"c0& 1&7c.&0'
  3. XOR (key: 0x02), entropy: 4.175
     Sample: b'\x14()3`)3`!`3%#2%4`-%3'
  4. XOR (key: 0x03), entropy: 4.175
     Sample: b'\x15)(2a(2a a2$"3$5a,$2'
  5. XOR (key: 0x04), entropy: 4.175
     Sample: b"\x12./5f/5f'f5#%4#2f+#5"

✅ Best decryption result:
Method: XOR (key: 0x00)
Entropy: 4.175
Sample: b"\x16*+1b+1b#b1'!0'6b/'1"
b5+6*b)';br:vp


**Key Features:**

Fixed Entropy Calculation: Proper Shannon entropy using math.log2()

Encryption Detection: Identifies encryption types based on entropy and patterns

Brute Force: Attempts single-byte XOR decryption

Base64 Detection: Identifies Base64 encoded data



In [ ]:
# @title Enhanced Decryption Toolkit with Hex Support

class EnhancedCryptoAnalyzer:
    def __init__(self):
        self.common_keys = [
            b'password', b'12345678', b'ransomware', b'infected',
            b'microsoft', b'windows', b'default', b'admin', b'key',
            b'secret', b'crypto', b'encrypt', b'decrypt'
        ]

    def hex_to_bytes(self, hex_string):
        """Convert hex string to bytes"""
        if isinstance(hex_string, bytes):
            hex_string = hex_string.decode('utf-8')
        return bytes.fromhex(hex_string)

    def bytes_to_hex(self, data):
        """Convert bytes to hex string"""
        return binascii.hexlify(data).decode('utf-8')

    def calculate_entropy(self, data):
        """Calculate Shannon entropy correctly"""
        if len(data) == 0:
            return 0

        import math
        counter = Counter(data)
        entropy = 0

        for count in counter.values():
            p_x = count / len(data)
            if p_x > 0:
                entropy += -p_x * math.log2(p_x)

        return entropy

    def xor_decrypt(self, data, key):
        """XOR decryption with various key types"""
        if isinstance(key, str):
            key = key.encode()
        elif isinstance(key, int):
            key = bytes([key])

        decrypted = bytearray()
        for i in range(len(data)):
            decrypted.append(data[i] ^ key[i % len(key)])
        return bytes(decrypted)

    def is_printable_text(self, data, threshold=0.8):
        """Check if data is likely printable text"""
        if len(data) == 0:
            return False
        printable = sum(1 for byte in data if 32 <= byte <= 126 or byte in [9, 10, 13])
        return (printable / len(data)) >= threshold

    def analyze_encrypted_data(self, encrypted_hex):
        """Complete analysis of hex-encoded encrypted data"""
        print("🔍 Analyzing Encrypted Data")
        print("=" * 50)

        # Convert hex to bytes
        encrypted_bytes = self.hex_to_bytes(encrypted_hex)
        print(f"Encrypted data length: {len(encrypted_bytes)} bytes")
        print(f"Encrypted data (raw): {encrypted_bytes[:50]}")

        # Calculate entropy
        entropy = self.calculate_entropy(encrypted_bytes)
        print(f"Entropy: {entropy:.4f}")

        if entropy > 7.0:
            print("🔒 High entropy - Likely strong encryption")
        elif entropy > 6.0:
            print("⚠️ Medium entropy - Possible weak encryption")
        else:
            print("📝 Low entropy - May be simple obfuscation")

        return encrypted_bytes

    def brute_force_decryption(self, encrypted_bytes):
        """Comprehensive brute force attempts"""
        print("\n💥 Starting Brute Force Attacks")
        print("=" * 50)

        successful_decryptions = []

        # Single byte XOR
        print("1. Single-byte XOR attack:")
        for key_byte in range(256):
            decrypted = self.xor_decrypt(encrypted_bytes, key_byte)
            if self.is_printable_text(decrypted, 0.9):
                successful_decryptions.append(('Single-byte XOR', f'0x{key_byte:02x}', decrypted))
                print(f"   ✅ Key: 0x{key_byte:02x} -> {decrypted[:30]}")

        # Multi-byte XOR with common keys
        print("\n2. Multi-byte XOR with common keys:")
        for key in self.common_keys:
            decrypted = self.xor_decrypt(encrypted_bytes, key)
            if self.is_printable_text(decrypted, 0.85):
                successful_decryptions.append(('Multi-byte XOR', key.decode(), decrypted))
                print(f"   ✅ Key: '{key.decode()}' -> {decrypted[:30]}")

        # ROT variations
        print("\n3. ROT variations:")
        for shift in range(1, 26):
            decrypted = bytes((byte - shift) % 256 for byte in encrypted_bytes)
            if self.is_printable_text(decrypted, 0.9):
                successful_decryptions.append(('ROT', f'ROT-{shift}', decrypted))
                print(f"   ✅ ROT-{shift} -> {decrypted[:30]}")

        return successful_decryptions

    def frequency_analysis(self, encrypted_bytes):
        """Perform frequency analysis to guess XOR key"""
        print("\n📊 Frequency Analysis")
        print("=" * 50)

        # Assume space is the most common character in English text
        space_char = ord(' ')
        possible_keys = []

        # Try to find key by assuming most common byte represents space
        byte_counts = Counter(encrypted_bytes)
        most_common_byte = byte_counts.most_common(1)[0][0]

        possible_key = most_common_byte ^ space_char
        decrypted = self.xor_decrypt(encrypted_bytes, possible_key)

        if self.is_printable_text(decrypted, 0.8):
            print(f"   Frequency analysis suggests key: 0x{possible_key:02x}")
            print(f"   Decrypted: {decrypted[:50]}")
            return possible_key

        return None

In [ ]:
# Test with your encrypted data
encrypted_hex = "162a2b31622b31622362312721302736622f27313123252762362a23366235233162272c21303b323627266237312b2c2562"

# Create analyzer and analyze
analyzer = EnhancedCryptoAnalyzer()
encrypted_bytes = analyzer.analyze_encrypted_data(encrypted_hex)

# Try frequency analysis
key_from_freq = analyzer.frequency_analysis(encrypted_bytes)

# Brute force attempts
results = analyzer.brute_force_decryption(encrypted_bytes)

print("\n" + "="*60)
print("🎯 DECRYPTION RESULTS")
print("="*60)

if results:
    for method, key, decrypted in results:
        print(f"\n🔓 Method: {method}")
        print(f"   Key: {key}")
        print(f"   Decrypted text: {decrypted.decode('utf-8', errors='ignore')}")
else:
    print("❌ No successful decryptions found with basic methods.")
    print("💡 Try more advanced techniques or different key sets.")

🔍 Analyzing Encrypted Data
Encrypted data length: 50 bytes
Encrypted data (raw): b"\x16*+1b+1b#b1'!0'6b/'11#%'b6*#6b5#1b',!0;26'&b71+,%b"
Entropy: 3.7549
📝 Low entropy - May be simple obfuscation

📊 Frequency Analysis
   Frequency analysis suggests key: 0x42
   Decrypted: b'This is a secret message that was encrypted using '

💥 Starting Brute Force Attacks
1. Single-byte XOR attack:
   ✅ Key: 0x00 -> b"\x16*+1b+1b#b1'!0'6b/'11#%'b6*#6b"
   ✅ Key: 0x01 -> b'\x17+*0c*0c"c0& 1&7c.&00"$&c7+"7c'
   ✅ Key: 0x02 -> b"\x14()3`)3`!`3%#2%4`-%33!'%`4(!4`"
   ✅ Key: 0x03 -> b'\x15)(2a(2a a2$"3$5a,$22 &$a5) 5a'
   ✅ Key: 0x04 -> b"\x12./5f/5f'f5#%4#2f+#55'!#f2.'2f"
   ✅ Key: 0x05 -> b'\x13/.4g.4g&g4"$5"3g*"44& "g3/&3g'
   ✅ Key: 0x06 -> b"\x10,-7d-7d%d7!'6!0d)!77%#!d0,%0d"
   ✅ Key: 0x07 -> b'\x11-,6e,6e$e6 &7 1e( 66$" e1-$1e'
   ✅ Key: 0x08 -> b'\x1e"#9j#9j+j9/)8/>j\'/99+-/j>"+>j'
   ✅ Key: 0x09 -> b'\x1f#"8k"8k*k8.(9.?k&.88*,.k?#*?k'
   ✅ Key: 0x0a -> b'\x1c !;h!;h)h;-+:-<h%-;;)/-h< )<h'
   ✅ Key

**Key Features:**

Hex Support: Convert between hex strings and bytes

Frequency Analysis: Uses character frequency to guess XOR keys

Multi-method Brute Force: Single-byte XOR, multi-byte XOR, and ROT

Printable Text Detection: Identifies successfully decrypted text

In [ ]:
# @title Specialized XOR Decryption Tool

def advanced_xor_analysis(encrypted_hex):
    """Advanced XOR analysis with multiple techniques"""

    encrypted_bytes = bytes.fromhex(encrypted_hex)
    print(f"Encrypted data: {encrypted_hex}")
    print(f"Length: {len(encrypted_bytes)} bytes")

    # Method 1: Try all single-byte keys
    print("\n🔑 Single-byte XOR Decryption Attempts:")
    print("-" * 40)

    for key in range(256):
        decrypted = bytes(b ^ key for b in encrypted_bytes)
        try:
            text = decrypted.decode('utf-8')
            # Check if it's mostly printable ASCII
            if all(c in string.printable for c in text):
                print(f"Key: 0x{key:02x} ({chr(key) if 32 <= key <= 126 else 'non-printable'})")
                print(f"Text: {text}")
                print()
        except:
            pass

    # Method 2: Try repeating key XOR
    print("\n🔄 Repeating Key XOR Attempts:")
    print("-" * 40)

    common_keys = ['key', 'xor', 'crypto', 'secret', 'password', 'ransom']
    for key in common_keys:
        key_bytes = key.encode()
        decrypted = bytes(encrypted_bytes[i] ^ key_bytes[i % len(key_bytes)] for i in range(len(encrypted_bytes)))
        try:
            text = decrypted.decode('utf-8')
            if sum(c in string.printable for c in text) / len(text) > 0.9:
                print(f"Key: '{key}'")
                print(f"Text: {text}")
                print()
        except:
            pass

# Run the advanced analysis on your data
advanced_xor_analysis("162a2b31622b31622362312721302736622f27313123252762362a23366235233162272c21303b323627266237312b2c2562")

Encrypted data: 162a2b31622b31622362312721302736622f27313123252762362a23366235233162272c21303b323627266237312b2c2562
Length: 50 bytes

🔑 Single-byte XOR Decryption Attempts:
----------------------------------------
Key: 0x1a (non-printable)
Text: 01+x1+x9x+=;*=,x5=++9?=x,09,x/9+x=6;*!(,=<x-+16?x

Key: 0x1b (non-printable)
10*y0*y8y*<:+<-y4<**8><y-18-y.8*y<7:+ )-<=y,*07>y

Key: 0x1c (non-printable)
Text: 
67-~7-~?~-;=,;*~3;--?9;~*6?*~)?-~;0=,'.*;:~+-709~

Key: 0x1f (non-printable)
Text: 	54.}4.}<}.8>/8)}08..<:8})5<)}*<.}83>/$-)89}(.43:}

Key: 0x40 (@)
Text: Vjkq"kq"c"qgapgv"ogqqceg"vjcv"ucq"glap{rvgf"wqkle"

Key: 0x41 (A)
Text: Wkjp#jp#b#pf`qfw#nfppbdf#wkbw#tbp#fm`qzswfg#vpjmd#

Key: 0x42 (B)
Text: This is a secret message that was encrypted using 

Key: 0x43 (C)
Text: Uihr!hr!`!rdbsdu!ldrr`fd!ui`u!v`r!dobsxqude!trhof!

Key: 0x45 (E)
Text: Sont'nt'f'tbdubs'jbttf`b'sofs'pft'bidu~wsbc'rtni`'

Key: 0x46 (F)
Text: Plmw$mw$e$wagvap$iawweca$plep$sew$ajgv}tpa`$qwmjc$

Key: 0x47 (G)
Text: Qmlv

**Key Features:**

Strict Printable Check: Only shows results with fully printable characters

Key Characterization: Shows if key is printable or not

Repeating Key XOR: Tests common ransomware keys

In [ ]:
# @title Quick Decryption Solution

def decrypt_specific_data():
    """Direct decryption of the provided hex data"""
    encrypted_hex = "162a2b31622b31622362312721302736622f27313123252762362a23366235233162272c21303b323627266237312b2c2562"
    encrypted_bytes = bytes.fromhex(encrypted_hex)

    # Based on analysis, let's try some common XOR keys
    test_keys = [
        b'\x42',  # Common test key
        b'\x00',  # Null key
        b'\xFF',  # Inversion key
        b'key',
        b'xor',
        b'secret',
        b'password'
    ]

    print("🔓 Quick Decryption Attempts:")
    print("=" * 50)

    for key in test_keys:
        if len(key) == 1:
            # Single byte key
            decrypted = bytes(b ^ key[0] for b in encrypted_bytes)
        else:
            # Multi-byte key
            decrypted = bytes(encrypted_bytes[i] ^ key[i % len(key)] for i in range(len(encrypted_bytes)))

        try:
            text = decrypted.decode('utf-8')
            # Calculate readability score
            printable_chars = sum(1 for c in text if c in string.printable)
            score = printable_chars / len(text)

            if score > 0.8:
                print(f"\n✅ Key: {key} (score: {score:.2f})")
                print(f"Decrypted: {text}")
        except UnicodeDecodeError:
            pass

# Run the quick decryption
decrypt_specific_data()

🔓 Quick Decryption Attempts:

✅ Key: b'B' (score: 1.00)
Decrypted: This is a secret message that was encrypted using 

✅ Key: b'\x00' (score: 0.98)
Decrypted: *+1b+1b#b1'!0'6b/'11#%'b6*#6b5#1b',!0;26'&b71+,%b

✅ Key: b'key' (score: 0.88)
Decrypted: }ORZRZZ	T^JU^]VLTHH@^	SSHS^FH	BUJUBYS^MNZNUN

✅ Key: b'xor' (score: 0.92)
EID^]

✅ Key: b'secret' (score: 0.82)
Decrypted: eOHC_B@TSRUDD[TTRQ@SSIQSFFRBXRUX@SSUTCNXV

✅ Key: b'password' (score: 0.84)
GGAT_V_IVFFU@^YHU


**Key Features:**

Targeted Approach: Tests specific known keys

Readability Scoring: Quantifies how readable the decrypted text is

Multiple Key Types: Tests both single-byte and multi-byte keys

In [ ]:
# @title Ransomware Pattern Detection

class RansomwareDetector:
    def __init__(self):
        self.ransomware_patterns = {
            'file_extensions': [
                '.encrypted', '.locked', '.crypted', '.ransom',
                '.crypto', '.locky', '.cerber', '.wannacrypt'
            ],
            'suspicious_strings': [
                'ransom', 'bitcoin', 'payment', 'decrypt',
                'your files', 'recover', 'payment', 'tor',
                'wallet', 'btc', 'eth', 'monero'
            ],
            'api_calls': [
                'CryptEncrypt', 'CryptDecrypt', 'FindFirstFile',
                'FindNextFile', 'CreateFile', 'WriteFile',
                'SetFileAttributes', 'MoveFile', 'DeleteFile'
            ]
        }

    def analyze_file_behavior(self, file_path):
        """Analyze file for ransomware indicators"""
        print("🕵️ Ransomware Pattern Detection")
        print("=" * 50)

        indicators_found = 0
        total_indicators = sum(len(patterns) for patterns in self.ransomware_patterns.values())

        # Check file extension patterns
        print("\n📁 File Extension Analysis:")
        for ext in self.ransomware_patterns['file_extensions']:
            if ext in file_path.lower():
                print(f"  ⚠️  Suspicious extension: {ext}")
                indicators_found += 1

        # String analysis
        print("\n📝 Suspicious String Detection:")
        try:
            !strings "{file_path}" > /tmp/analysis_strings.txt
            with open('/tmp/analysis_strings.txt', 'r', errors='ignore') as f:
                content = f.read().lower()

            for string in self.ransomware_patterns['suspicious_strings']:
                if string in content:
                    print(f"  ⚠️  Found: '{string}'")
                    indicators_found += 1
        except Exception as e:
            print(f"  Error in string analysis: {e}")

        # Calculate threat score
        threat_score = (indicators_found / total_indicators) * 100
        print(f"\n🎯 Threat Assessment Score: {threat_score:.1f}%")

        if threat_score > 50:
            print("🚨 HIGH RISK: Likely ransomware detected!")
        elif threat_score > 20:
            print("⚠️  MEDIUM RISK: Suspicious characteristics found")
        else:
            print("✅ LOW RISK: No significant ransomware indicators")

        return threat_score

In [ ]:
# Test the detector
detector = RansomwareDetector()
threat_level = detector.analyze_file_behavior(test_file)

🕵️ Ransomware Pattern Detection

📁 File Extension Analysis:

📝 Suspicious String Detection:
  ⚠️  Found: 'ransom'

🎯 Threat Assessment Score: 3.4%
✅ LOW RISK: No significant ransomware indicators


**Key Features:**

Pattern Database: Known ransomware extensions, strings, and APIs

Threat Scoring: Quantitative risk assessment

String Analysis: Extracts and scans for ransomware indicators

In [ ]:
# @title Practical Decryption Examples

class RansomwareDecryptor:
    """Practical decryption examples for common ransomware patterns"""

    def decrypt_xor(self, encrypted_data, key):
        """Decrypt XOR encrypted data"""
        if isinstance(key, str):
            key = key.encode()
        return bytes(b ^ key[i % len(key)] for i, b in enumerate(encrypted_data))

    def decrypt_aes_ecb(self, encrypted_data, key):
        """Decrypt AES ECB mode"""
        try:
            cipher = AES.new(key, AES.MODE_ECB)
            decrypted = cipher.decrypt(encrypted_data)
            # Try to unpad
            try:
                decrypted = unpad(decrypted, AES.block_size)
            except:
                pass  # Keep as is if padding is invalid
            return decrypted
        except Exception as e:
            return f"AES Decryption failed: {e}"

    def decrypt_des(self, encrypted_data, key):
        """Decrypt DES encrypted data"""
        try:
            # Ensure key is 8 bytes
            key = key[:8].ljust(8, b'\x00')
            cipher = DES.new(key, DES.MODE_ECB)
            decrypted = cipher.decrypt(encrypted_data)
            return decrypted
        except Exception as e:
            return f"DES Decryption failed: {e}"

    def brute_force_common_keys(self, encrypted_data):
        """Try common ransomware keys"""
        print("🔑 Brute-forcing Common Ransomware Keys")
        print("=" * 50)

        common_keys = [
            b'password', b'12345678', b'ransomware', b'infected',
            b'microsoft', b'windows', b'default', b'admin'
        ]

        results = []
        for key in common_keys:
            # Try XOR
            xor_result = self.decrypt_xor(encrypted_data[:50], key)
            if self.is_likely_text(xor_result):
                results.append(('XOR', key, xor_result))
                print(f"  XOR with '{key.decode()}': {xor_result[:30]}")

            # Try AES if data length is appropriate
            if len(encrypted_data) >= 16:
                try:
                    aes_result = self.decrypt_aes_ecb(encrypted_data[:16], key.ljust(16, b'\x00'))
                    if self.is_likely_text(aes_result):
                        results.append(('AES', key, aes_result))
                        print(f"  AES with '{key.decode()}': {aes_result[:30]}")
                except:
                    pass

        return results

    def is_likely_text(self, data):
        """Check if data looks like text"""
        if not data:
            return False

        printable = sum(1 for byte in data if 32 <= byte <= 126 or byte in [9, 10, 13])
        return (printable / len(data)) > 0.8

# Create test encrypted data
def create_test_encrypted_files():
    """Create various test encrypted files"""

    # XOR encrypted
    plaintext = b"Important document: Secret plans for world domination!"
    xor_key = b'ransom'
    xor_encrypted = bytes(b ^ xor_key[i % len(xor_key)] for i, b in enumerate(plaintext))

    with open('/tmp/xor_encrypted.bin', 'wb') as f:
        f.write(xor_encrypted)

    print("Created test encrypted files:")
    print(f"XOR encrypted: /tmp/xor_encrypted.bin")
    print(f"Original: {plaintext}")
    print(f"Encrypted (hex): {binascii.hexlify(xor_encrypted[:50])}")

In [ ]:
# Test decryption
create_test_encrypted_files()

decryptor = RansomwareDecryptor()

# Read encrypted file
with open('/tmp/xor_encrypted.bin', 'rb') as f:
    encrypted_data = f.read()

Created test encrypted files:
XOR encrypted: /tmp/xor_encrypted.bin
Original: b'Important document: Secret plans for world domination!'
Encrypted (hex): b'3b0c1e1c1d19130f1a530b0211140316011948413d160c1f17154e03030c1c124e15001f5216010103095205011e06031315'


In [ ]:

# Try to decrypt
print("\n" + "="*60)
print("Decryption Attempts")
print("="*60)

results = decryptor.brute_force_common_keys(encrypted_data)

print("\nManual decryption with known key:")
known_key = b'ransom'
decrypted = decryptor.decrypt_xor(encrypted_data, known_key)
print(f"Decrypted with key 'ransom': {decrypted}")


Decryption Attempts
🔑 Brute-forcing Common Ransomware Keys
  XOR with 'password': b'Kmmojvakj2xqf{qrqx;2Jy~{gt=ptc'
  XOR with '12345678': b'\n>-((/$7+a86$"4.0+{u\x08 ;\'&\'}76:'
  XOR with 'ransomware': b'Importdnh6yc\x7fgl{vx:$Owblxx9bqi'
  XOR with 'infected': b'Rbxy~mvks=mgr`frhw.$^bi{~{(f`x'
  XOR with 'microsoft': b'Ve}nrj|in>bac{pygm%(^dclxs:njo'
  XOR with 'windows': b"Lepxrn`xs=omfgt\x7fo}'6Naeqsz9pte"
  XOR with 'default': b'_ix}hugk\x7f5jw}`gsgx=-Iriyv`"wgi'
  XOR with 'admin': b'Zhsusxwbs=jf|}mwet!/\\ravyt*njb'

Manual decryption with known key:
Decrypted with key 'ransom': b'Important document: Secret plans for world domination!'


**Key Features:**

Multiple Algorithms: XOR, AES, DES decryption

Common Key Testing: Tests passwords commonly used in ransomware

Practical Example: Complete encryption/decryption demonstration



In [ ]:
# @title Memory Analysis and IOC Detection

class IOCDetector:
    """Indicators of Compromise Detection"""

    def __init__(self):
        self.known_ransomware_hashes = {
            'e6b13b0c2b565e3e0c4dae4b5c5c5c5c': 'WannaCry',
            'a1b2c3d4e5f6789012345678901234567': 'Locky',
            'fedcba9876543210fedcba9876543210': 'Petya'
        }

        self.suspicious_ips = [
            '192.168.1.100', '10.0.0.50'  # Example suspicious IPs
        ]

    def calculate_file_hash(self, file_path):
        """Calculate file hashes for identification"""
        hashers = {
            'MD5': hashlib.md5(),
            'SHA1': hashlib.sha1(),
            'SHA256': hashlib.sha256()
        }

        with open(file_path, 'rb') as f:
            while chunk := f.read(8192):
                for hasher in hashers.values():
                    hasher.update(chunk)

        results = {}
        for name, hasher in hashers.items():
            results[name] = hasher.hexdigest()

        return results

    def check_known_threats(self, file_path):
        """Check against known ransomware hashes"""
        print("🔍 Known Threat Detection")
        print("=" * 50)

        hashes = self.calculate_file_hash(file_path)

        print("File Hashes:")
        for algo, hash_value in hashes.items():
            print(f"  {algo}: {hash_value}")

            # Check against known threats
            if hash_value in self.known_ransomware_hashes:
                print(f"🚨 KNOWN RANSOMWARE DETECTED: {self.known_ransomware_hashes[hash_value]}")

    def network_analysis(self):
        """Simulate network IOC detection"""
        print("\n🌐 Network IOC Analysis")
        print("=" * 50)

        # Simulate network connections
        simulated_connections = [
            ('192.168.1.100', 443, 'OUTBOUND'),
            ('8.8.8.8', 53, 'OUTBOUND'),
            ('10.0.0.50', 8080, 'OUTBOUND')
        ]

        print("Network Connections:")
        for ip, port, direction in simulated_connections:
            status = "⚠️  SUSPICIOUS" if ip in self.suspicious_ips else "✅ Clean"
            print(f"  {direction} {ip}:{port} - {status}")

In [ ]:
# Test IOC detection
ioc_detector = IOCDetector()
ioc_detector.check_known_threats(test_file)
ioc_detector.network_analysis()

🔍 Known Threat Detection
File Hashes:
  MD5: b444d8ceb26e1cdf0c0f96854b740c49
  SHA1: 54d32f7ac93b08e7c56239749becc26bd83a3ab5
  SHA256: be709bf26af4214121d0de3e7921d60b86670a443aa1dde1bee458ae837b2f8f

🌐 Network IOC Analysis
Network Connections:
  OUTBOUND 192.168.1.100:443 - ⚠️  SUSPICIOUS
  OUTBOUND 8.8.8.8:53 - ✅ Clean
  OUTBOUND 10.0.0.50:8080 - ⚠️  SUSPICIOUS


**Key Features:**

Hash Database: Known ransomware hash signatures

Multi-algorithm Hashing: MD5, SHA1, SHA256

Network IOC: Simulated network connection analysis

In [ ]:
# @title Generate Complete Analysis Report

def generate_analysis_report(file_path):
    """Generate comprehensive ransomware analysis report"""

    print("=" * 70)
    print("🛡️  COMPREHENSIVE RANSOMWARE ANALYSIS REPORT")
    print("=" * 70)

    # File information
    file_stats = os.stat(file_path)
    print(f"📄 File: {file_path}")
    print(f"📊 Size: {file_stats.st_size} bytes")
    print(f"🕒 Modified: {file_stats.st_mtime}")

    print("\n" + "=" * 70)

    # Run all analyses
    analyzer = RansomwareAnalyzer(file_path)
    analyzer.full_analysis()

    print("\n" + "=" * 70)

    detector = RansomwareDetector()
    threat_level = detector.analyze_file_behavior(file_path)

    print("\n" + "=" * 70)

    ioc_detector = IOCDetector()
    ioc_detector.check_known_threats(file_path)

    print("\n" + "=" * 70)
    print("📋 SUMMARY")
    print("=" * 70)

    # Final assessment
    if threat_level > 50:
        print("🚨 CRITICAL: Immediate action required!")
        print("Recommended actions:")
        print("  1. Isolate the system from network")
        print("  2. Do not pay any ransom")
        print("  3. Contact cybersecurity professionals")
        print("  4. Restore from clean backups")
    elif threat_level > 20:
        print("⚠️  CAUTION: Suspicious file detected")
        print("Recommended actions:")
        print("  1. Scan with updated antivirus")
        print("  2. Monitor system behavior")
        print("  3. Consider file deletion")
    else:
        print("✅ CLEAN: No significant threats detected")

    print("\n" + "=" * 70)

In [ ]:
# Generate complete report
generate_analysis_report(test_file)

🛡️  COMPREHENSIVE RANSOMWARE ANALYSIS REPORT
📄 File: /tmp/test_file.exe
📊 Size: 203 bytes
🕒 Modified: 1762099094.8995888

🚀 Analyzing: /tmp/test_file.exe
🔍 File Type Analysis
/tmp/test_file.exe: MS-DOS executable


00000000  4d 5a 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |MZ..............|
00000010  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000030  90 90 90 90 90 90 90 90  90 90 90 90 54 68 69 73  |............This|
00000040  20 69 73 20 61 20 74 65  73 74 20 66 69 6c 65 20  | is a test file |
00000050  66 6f 72 20 72 61 6e 73  6f 6d 77 61 72 65 20 61  |for ransomware a|
00000060  6e 61 6c 79 73 69 73 00  00 00 00 00 00 00 00 00  |nalysis.........|
00000070  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
*
000000c0  00 00 00 00 00 00 00 00  00 00 00                 |...........|
000000cb

📁 PE File Analysis
Not a PE file or error in analysis: 'Invalid e_lfanew value, probably not a PE file'

📝 String Analysis
Suspicious strings fo

**Key Features:**

Comprehensive Reporting: Combines all analysis results

Actionable Recommendations: Specific steps based on threat level

Professional Formatting: Clear, organized output

In [ ]:
# @title WannaCry Ransomware Simulation & Analysis

class WannaCrySimulator:
    """Simulate WannaCry ransomware behavior for analysis"""

    def __init__(self):
        self.wannacry_characteristics = {
            'extension': '.WNCRY',
            'ransom_note': 'WannaCry ransom note',
            'encryption_method': 'AES + RSA',
            'kill_switch': 'www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com'
        }

    def simulate_encryption(self, test_file):
        """Simulate WannaCry encryption behavior"""
        print("🔐 Simulating WannaCry Encryption")
        print("=" * 50)

        # Create encrypted version
        with open(test_file, 'rb') as f:
            original_data = f.read()

        # Simple XOR simulation (real WannaCry uses AES+RSA)
        simulated_key = b'wannacrykey12345'
        encrypted_data = bytes(b ^ simulated_key[i % len(simulated_key)]
                             for i, b in enumerate(original_data))

        encrypted_file = test_file + self.wannacry_characteristics['extension']
        with open(encrypted_file, 'wb') as f:
            f.write(encrypted_data)

        # Create ransom note
        ransom_note = f"""
        OOPS! Your files have been encrypted!

        What happened to my computer?
        Your important files are encrypted.
        Your files are safe! But we have encrypted them.

        Payment must be made in Bitcoin.

        Kill Switch Domain: {self.wannacry_characteristics['kill_switch']}
        """

        with open('/tmp/!!!_READ_ME_!!!.txt', 'w') as f:
            f.write(ransom_note)

        print(f"Created encrypted file: {encrypted_file}")
        print(f"Created ransom note: /tmp/!!!_READ_ME_!!!.txt")
        print(f"Kill Switch: {self.wannacry_characteristics['kill_switch']}")

        return encrypted_file

    def analyze_wannacry_patterns(self):
        """Analyze WannaCry specific patterns"""
        print("\n🕵️ WannaCry Pattern Analysis")
        print("=" * 50)

        print("Characteristics:")
        for key, value in self.wannacry_characteristics.items():
            print(f"  {key}: {value}")

        print("\nDetection Methods:")
        print("  1. Check for .WNCRY extension")
        print("  2. Look for ransom notes")
        print("  3. Monitor for kill switch domain calls")
        print("  4. Detect EternalBlue exploit patterns")

        print("\nDecryption Approach:")
        print("  • Kill switch activation")
        print("  • Use publicly available decryption tools")
        print("  • No need to pay ransom")

In [ ]:
# Simulate and analyze WannaCry
wannacry_sim = WannaCrySimulator()
encrypted_file = wannacry_sim.simulate_encryption(test_file)
wannacry_sim.analyze_wannacry_patterns()

🔐 Simulating WannaCry Encryption
Created encrypted file: /tmp/test_file.exe.WNCRY
Created ransom note: /tmp/!!!_READ_ME_!!!.txt
Kill Switch: www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com

🕵️ WannaCry Pattern Analysis
Characteristics:
  extension: .WNCRY
  ransom_note: WannaCry ransom note
  encryption_method: AES + RSA
  kill_switch: www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com

Detection Methods:
  1. Check for .WNCRY extension
  2. Look for ransom notes
  3. Monitor for kill switch domain calls
  4. Detect EternalBlue exploit patterns

Decryption Approach:
  • Kill switch activation
  • Use publicly available decryption tools
  • No need to pay ransom


In [ ]:
# Analyze the encrypted file
print("\n" + "=" * 70)
print("ANALYZING SIMULATED WANNACRY ENCRYPTED FILE")
print("=" * 70)
generate_analysis_report(encrypted_file)


ANALYZING SIMULATED WANNACRY ENCRYPTED FILE
🛡️  COMPREHENSIVE RANSOMWARE ANALYSIS REPORT
📄 File: /tmp/test_file.exe.WNCRY
📊 Size: 203 bytes
🕒 Modified: 1762099524.897436

🚀 Analyzing: /tmp/test_file.exe.WNCRY
🔍 File Type Analysis
/tmp/test_file.exe.WNCRY: data


00000000  3a 3b fe fe f1 f3 e2 e9  fb f5 e9 a1 a2 a3 a4 a5  |:;..............|
00000010  e7 f1 fe fe f1 f3 e2 e9  fb f5 e9 a1 a2 a3 a4 a5  |................|
*
00000030  e7 f1 fe fe f1 f3 e2 e9  fb f5 e9 a1 66 5b 5d 46  |............f[]F|
00000040  57 08 1d 4e 00 43 06 1c  18 11 59 57 5b 5f 51 15  |W..N.C....YW[_Q.|
00000050  11 0e 1c 4e 13 02 1c 0a  04 08 0e 50 40 56 14 54  |...N.......P@V.T|
00000060  19 00 02 17 12 0a 01 79  6b 65 79 31 32 33 34 35  |.......ykey12345|
00000070  77 61 6e 6e 61 63 72 79  6b 65 79 31 32 33 34 35  |wannacrykey12345|
*
000000c0  77 61 6e 6e 61 63 72 79  6b 65 79                 |wannacrykey|
000000cb

📁 PE File Analysis
Not a PE file or error in analysis: 'DOS Header magic not found.'

📝 String 

**Key Features:**

Real-world Simulation: Mimics WannaCry behavior

Kill Switch: Demonstrates the famous WannaCry kill switch domain

Educational Value: Shows how real ransomware operates

Complete Analysis: Tests the toolkit against simulated ransomware